In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
# 下载训练集
train_dataset = datasets.MNIST(root='./',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
# 下载测试集
test_dataset = datasets.MNIST(root='./',
                                train=False,
                                transform=transforms.ToTensor(),
                                download=True)

In [5]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=True)
# 装载测试集
test_loader = DataLoader(dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=True)

In [6]:
for i, data in enumerate(train_loader):
    # 获得数据和对应的标签
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [15]:
# 定义网络结构
# input_size输入特征的大小
# hidden_size，LSTM模块的数量
# num_layers，LSTM的层数
# LSTM默认input(seq_len, batch, feature)
# batch_first=True,input和output(batch, seq_len, feature)
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(
            input_size=28,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )
        self.out = torch.nn.Linear(in_features=64,out_features=10)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        # (batch, seq_len, feature)
        x = x.view(-1,28,28)
        # output:[batch, seq_len, hidden_size]包含每个序列的输出结果
        # 虽然LSTM的batch_first为True，但是h_n,c_n的第0个维度还是num_layers
        # h_n:[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        # c_n:[num_layers, batch, hidden_size]只包含最后一个序列的输出结果
        output,(h_n,c_n) = self.lstm(x)
        output_in_last_timestep = h_n[-1,:,:]
        x = self.out(output_in_last_timestep)
        x = self.softmax(x)
        return x

In [16]:
LR = 0.0003
# 定义模型
model = LSTM()
# 定义代价函数
entropy_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), LR)

In [17]:
def train():
    model.train()
    for i, data in enumerate(train_loader):
        # 获得数据和对应的标签
        inputs, labels = data
        # 获得模型预测结果，（64，10）
        out = model(inputs)
        # 交叉熵代价函数out(batch,C),labels(batch)
        loss = entropy_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()


def test():
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得数据和对应的标签
        inputs, labels = data
        # 获得模型预测结果
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc: {0}".format(correct.item() / len(test_dataset)))

    correct = 0
    for i, data in enumerate(train_loader):
        # 获得数据和对应的标签
        inputs, labels = data
        # 获得模型预测结果
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Train acc: {0}".format(correct.item() / len(train_dataset)))

In [19]:
for epoch in range(0, 10):
    print('epoch:',epoch)
    train()
    test()

epoch: 0
Test acc: 0.9624
Train acc: 0.9619833333333333
epoch: 1
Test acc: 0.9625
Train acc: 0.9631666666666666
epoch: 2
Test acc: 0.9651
Train acc: 0.9664666666666667
epoch: 3
Test acc: 0.965
Train acc: 0.9690666666666666
epoch: 4
Test acc: 0.9609
Train acc: 0.9656833333333333
epoch: 5
Test acc: 0.9671
Train acc: 0.9700333333333333
epoch: 6
Test acc: 0.9679
Train acc: 0.9705666666666667
epoch: 7
Test acc: 0.9694
Train acc: 0.9724833333333334
epoch: 8
Test acc: 0.9644
Train acc: 0.9684666666666667
epoch: 9
Test acc: 0.9658
Train acc: 0.97045
